In [2]:
import numpy as np
from collections import defaultdict
import heapq

def dijkstra(graph, start):
    distances = {node: float('infinity') for node in graph}
    distances[start] = 0
    pq = [(0, start)]
    visited = set()
    
    while pq:
        current_distance, current_vertex = heapq.heappop(pq)
        
        if current_vertex in visited:
            continue
            
        visited.add(current_vertex)
        
        for neighbor, weight in graph[current_vertex].items():
            distance = current_distance + weight
            
            if distance < distances[neighbor]:
                distances[neighbor] = distance
                heapq.heappush(pq, (distance, neighbor))
                
    return distances

def hungarian_algorithm(cost_matrix):
    """
    Implementação do Algoritmo Húngaro (Munkres)
    """
    n, m = cost_matrix.shape
    
    # Passo 1: Subtrair o menor elemento de cada linha
    cost_matrix = cost_matrix.copy()
    for i in range(n):
        cost_matrix[i] = cost_matrix[i] - np.min(cost_matrix[i])
    
    # Passo 2: Subtrair o menor elemento de cada coluna
    for j in range(m):
        cost_matrix[:,j] = cost_matrix[:,j] - np.min(cost_matrix[:,j])
    
    # Passo 3: Cobrir todos os zeros com o número mínimo de linhas
    def find_zero_locations(matrix):
        return np.where(matrix == 0)
    
    def find_min_lines():
        covered_rows = set()
        covered_cols = set()
        
        # Encontrar zeros na matriz
        row_ind, col_ind = find_zero_locations(cost_matrix)
        zero_positions = list(zip(row_ind, col_ind))
        
        # Marcar linhas que têm zeros únicos em suas colunas
        col_zeros = defaultdict(list)
        for r, c in zero_positions:
            col_zeros[c].append(r)
            
        for col, rows in col_zeros.items():
            if len(rows) == 1:
                covered_rows.add(rows[0])
        
        # Marcar colunas que têm zeros em linhas não marcadas
        for r, c in zero_positions:
            if r not in covered_rows:
                covered_cols.add(c)
        
        return covered_rows, covered_cols
    
    while True:
        covered_rows, covered_cols = find_min_lines()
        if len(covered_rows) + len(covered_cols) >= n:
            break
            
        # Encontrar o menor elemento não coberto
        min_val = float('infinity')
        for i in range(n):
            for j in range(m):
                if i not in covered_rows and j not in covered_cols:
                    min_val = min(min_val, cost_matrix[i,j])
        
        # Subtrair o menor elemento não coberto
        for i in range(n):
            for j in range(m):
                if i not in covered_rows and j not in covered_cols:
                    cost_matrix[i,j] -= min_val
                elif i in covered_rows and j in covered_cols:
                    cost_matrix[i,j] += min_val
    
    # Encontrar a atribuição ótima
    assignment = []
    row_ind, col_ind = find_zero_locations(cost_matrix)
    used_cols = set()
    
    for r, c in zip(row_ind, col_ind):
        if c not in used_cols:
            assignment.append((r, c))
            used_cols.add(c)
    
    return assignment

def solve_polish_army_problem():
    # Criar o grafo
    graph = defaultdict(dict)
    
    # Adicionar arestas do grafo
    edges = [
        ('α1', '1', 1), ('α1', '2', 3),
        ('α2', '1', 2), ('α2', '2', 3),
        ('α3', '2', 1), ('α3', 'β3', 4),
        ('1', '2', 1),
        ('1', 'β1', 2), ('1', 'β2', 4),
        ('2', 'β1', 3), ('2', 'β2', 5), ('2', 'β3', 2)
    ]
    
    for start, end, weight in edges:
        graph[start][end] = weight
        graph[end][start] = weight
    
    # Calcular distâncias mínimas
    origins = ['α1', 'α2', 'α3']
    destinations = ['β1', 'β1', 'β2', 'β3']  # β1 aparece duas vezes pois precisa de 2 divisões
    
    # Matriz de custos
    costs = []
    for origin in origins:
        distances = dijkstra(graph, origin)
        row = [distances[dest.replace('β1', 'β1')] for dest in destinations]
        costs.append(row)
    
    # Converter para numpy array
    cost_matrix = np.array(costs)
    
    # Aplicar o algoritmo húngaro
    assignment = hungarian_algorithm(cost_matrix)
    
    # Converter a atribuição para o formato original
    final_assignment = []
    for orig_idx, dest_idx in assignment:
        origin = origins[orig_idx]
        destination = destinations[dest_idx]
        final_assignment.append((origin, destination))
    
    # Calcular custo total
    total_cost = sum(cost_matrix[i,j] for i, j in assignment)
    
    return final_assignment, total_cost

# Executar a solução
assignment, total_cost = solve_polish_army_problem()

print("\nAtribuição ótima encontrada:")
for origin, dest in assignment:
    print(f"Mover divisão de {origin} para {dest}")
print(f"\nCusto total (risco total): {total_cost}")


Atribuição ótima encontrada:
Mover divisão de α1 para β1
Mover divisão de α1 para β1
Mover divisão de α1 para β2
Mover divisão de α3 para β3

Custo total (risco total): 14
